In [ ]:
import scipy.stats as sps
from sklearn.model_selection import train_test_split
import gc
from sklearn.linear_model import LinearRegression
from dateutil.parser import *
import pandas as pd
import numpy as np
import os
import yt.wrapper as yt
from collections import Counter
import json
import typing as tp
import ast
from datetime import datetime
os.chdir('/Users/lunin-dv/Desktop/Library/')
import importlib
import my_library as lib
import operator
import re
importlib.reload(lib)
os.chdir('/Users/lunin-dv/Desktop/data/')

# 1. Добавление

In [ ]:
req = """
select DISTINCT
    lead_id as task_id, 
    first_name, 
    last_name,
    account_name,
    phone_mobile, 
    email, 
    status,
    lead_source,
    lead_source_description,
    description,
    timezone
from 
    "//home/cloud_analytics/kulaga/leads_cube"
where 
    lead_source = 'cold_calls'
    and lower(tag_name) = 'прозвон_кц'
    and email not in (null, '')
    and toDate(date_modified) <= addDays(NOW(), -2)
    and toDate(date_modified) >= toDate('2020-07-29')
    and phone_mobile != ''
    AND isNotNull(phone_mobile)
FORMAT TabSeparatedWithNames
""".encode('utf-8')
df = lib.execute_query(req)

In [ ]:
df.shape

In [ ]:
old_df = lib.execute_query("""
SELECT 
    DISTINCT
    task_id
FROM "//home/cloud_analytics/lunin-dv/call_center_output_streaming_output/pcc_drop_task_id"
FORMAT TabSeparatedWithNames
""")

In [ ]:
df = df[~df["task_id"].isin(old_df["task_id"])]

In [ ]:
df.shape

In [ ]:
str(datetime.now()).replace(" ", "T")

In [ ]:
if len(df) > 0:
    lib.save_table(str(datetime.now()).replace(" ", "T"), 
               "//home/cloud_analytics/call_center/input/activation", df)

## dropped list update


In [ ]:
added_df = pd.DataFrame(df["task_id"])

In [ ]:
lib.save_table("pcc_drop_task_id", 
               "//home/cloud_analytics/lunin-dv/call_center_output_streaming_output", added_df, append=True)

# 2. Update Lead

In [ ]:
req = """
SELECT
    toInt64(NOW()) as Timestamp,
    CAST(task_id as String) as CRM_Lead_ID,
    if(activated == 1, ['activated'], ['not activated']) as tag_a,
    arrayReduce('groupUniqArray', arrayConcat(tag_b, tag_a)) as Tags,
    'In Process' as Status,
    toDateTime(toInt64(assignment_approve_time / 1000)) as Last_communication_date,
    Comment as Notes
FROM "//home/cloud_analytics/call_center/output/pcc_output" as a
INNER JOIN (
    SELECT
        DISTINCT
            lead_id,
            groupUniqArray(tag_name) as tag_b
    FROM "//home/cloud_analytics/kulaga/leads_cube"
    GROUP BY lead_id
) as b
ON a.task_id == b.lead_id
FORMAT TabSeparatedWithNames
""".encode('utf-8')

update_df = lib.execute_query(req)
update_df.drop(columns = ['tag_a'], inplace=True)

In [ ]:
update_df['Tags'] = update_df['Tags'].apply(lambda x: x if x.remove('прозвон_кц') else x )

In [ ]:
update_df['Tags'] = update_df['Tags'].astype(str)

In [ ]:
update_df['Tags'] = update_df['Tags'].apply(lambda x: x.replace("'", '"') if not pd.isnull(x) else np.nan)

In [ ]:
old_df = lib.execute_query("""
SELECT
    DISTINCT 
    CRM_Lead_ID || '__' || toString(ifNull(Last_communication_date, '')) as key
FROM "//home/cloud_analytics/export/crm/update_call_center_leads/update_leads"
FORMAT TabSeparatedWithNames
""")

In [ ]:
update_df = update_df[~update_df[['CRM_Lead_ID', 'Last_communication_date']]\
          .apply(lambda x: x['CRM_Lead_ID'] + "__" + x['Last_communication_date'], axis=1)\
          .isin(old_df['key'])]

In [ ]:
update_df.shape

In [ ]:
lib.save_table('update_leads', "//home/cloud_analytics/export/crm/update_call_center_leads",
               update_df, append=True)